In [ ]:
pip install cirq

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 47 kB 4.3 MB/s 
     |████████████████████████████████| 1.6 MB 8.8 MB/s 
     |████████████████████████████████| 437 kB 55.1 MB/s 
     |████████████████████████████████| 328 kB 58.8 MB/s 
     |████████████████████████████████| 97 kB 5.8 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 220 kB 66.0 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 145 kB 65.9 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 10.1 MB 25.5 MB/s 
     |████████████████████████████████| 65 kB 3.0 MB/s 
     |████████████████████████████████| 229 kB 54.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 243 kB 38.9 MB/s 
     |████████████████████████

In [ ]:
"""
Created on Thu Feb 24 21:54:47 2022

@author: Ananyo
"""
import cirq
import math
import random
import fractions
import time
import matplotlib.pyplot as plt
import sympy
from typing import Callable, Optional, Sequence, Union

"""This Code is capable of implementing the Classical and Quantum Period Finding Part of Shor's Algorithm"""
##############################################################################
#################################  SECTION 1  ################################
##############################################################################

"""Functions Implementing the Quantum Period Finding Section of Algorithm""" 

"""
    Class employs Quantum Modular Exponentiation (x^e mod(N)) implementation together 
    with the Quantum Phase Estimation to compute the order of x mod(N)
"""
class QModExp(cirq.ArithmeticOperation):

    def __init__(
        self,
        target: Sequence[cirq.Qid],
        exponent: Union[int, Sequence[cirq.Qid]],
        base: int,
        modulus: int,
    ) -> None:
        if len(target) < modulus.bit_length():
            raise ValueError(
                f'Register with {len(target)} qubits is too small for modulus {modulus}'
            )
        self.target = target
        self.exponent = exponent
        self.base = base
        self.modulus = modulus

    def registers(self) -> Sequence[Union[int, Sequence[cirq.Qid]]]:
        return self.target, self.exponent, self.base, self.modulus

    def with_registers(
        self,
        *new_registers: Union[int, Sequence['cirq.Qid']],) -> 'QModExp':
        if len(new_registers) != 4:
            raise ValueError(
                f'Expected 4 registers (target, exponent, base, '
                f'modulus), but got {len(new_registers)}'
            )
        target, exponent, base, modulus = new_registers
        if not isinstance(target, Sequence):
            raise ValueError(f'Target must be a qubit register, got {type(target)}')
        if not isinstance(base, int):
            raise ValueError(f'Base must be a classical constant, got {type(base)}')
        if not isinstance(modulus, int):
            raise ValueError(f'Modulus must be a classical constant, got {type(modulus)}')
        return QModExp(target, exponent, base, modulus)

    def apply(self, *register_values: int) -> int:
        assert len(register_values) == 4
        target, exponent, base, modulus = register_values
        if target >= modulus:
            return target
        return (target * base ** exponent) % modulus
    
    """
        Declaring Sybmols to be used for Circuit for Quantum Period Finding
        Can be uncommented in case the circuit needs to be printed
    """

    # def _circuit_diagram_info_(
    #     self,
    #     args: cirq.CircuitDiagramInfoArgs,
    # ) -> cirq.CircuitDiagramInfo:
    #     assert args.known_qubits is not None
    #     wire_symbols: List[str] = []
    #     t, e = 0, 0
    #     for qubit in args.known_qubits:
    #         if qubit in self.target:
    #             if t == 0:
    #                 if isinstance(self.exponent, Sequence):
    #                     e_str = 'e'
    #                 else:
    #                     e_str = str(self.exponent)
    #                 wire_symbols.append(f'QModExp(t*{self.base}**{e_str} % {self.modulus})')
    #             else:
    #                 wire_symbols.append('t' + str(t))
    #             t += 1
    #         if isinstance(self.exponent, Sequence) and qubit in self.exponent:
    #             wire_symbols.append('e' + str(e))
    #             e += 1
    #     return cirq.CircuitDiagramInfo(wire_symbols=tuple(wire_symbols))


"""
    Creating the Circuit for Quantum Period Finding 
        x: positive integer whose order modulo n is to be found
        N: modulus relative to which the order of x is to be found
    Returns:
        Quantum circuit for finding the order of x modulo N
"""
def quantum_period_finding_circuit(x: int, N: int) -> cirq.Circuit:
    L = N.bit_length()
    target = cirq.LineQubit.range(L)
    exponent = cirq.LineQubit.range(L, 3 * L + 3)
    return cirq.Circuit(
        cirq.X(target[L - 1]),
        cirq.H.on_each(*exponent),
        QModExp(target, exponent, x, N),
        cirq.qft(*exponent, inverse=True),
        cirq.measure(*exponent, key='exponent'),

    )


    """
       Function used to evaluate s/r where r is the order of x modulo N 
        x: integer whose order is to be computed, must be greater than one
           and belong to the multiplicative group of integers modulo n (which
           consists of positive integers relatively prime to n),
        n: modulus of the multiplicative group.
    Returns:
        Smallest positive integer r such that x**r == 1 mod n 

    """

def read_eigenphase(result: cirq.Result) -> float:

    exponent_as_integer = result.data['exponent'][0]
    exponent_num_bits = result.measurements['exponent'].shape[1]
    return float(exponent_as_integer / 2 ** exponent_num_bits)

    """
    """
def find_order(x: int, N: int) -> Optional[int]:

    if x < 2 or N <= x or math.gcd(x, N) > 1:
        raise ValueError(f'Invalid x={x} for modulus n={N}.')

    circuit = quantum_period_finding_circuit(x, N)
    result = cirq.sample(circuit)
    eigenphase = read_eigenphase(result)
    f = fractions.Fraction.from_float(eigenphase).limit_denominator(N)
    if f.numerator == 0:
        return None  # coverage: ignore
    r = f.denominator
    if x ** r % N != 1:
        return None  # coverage: ignore
    return r



##############################################################################
#################################  SECTION 2  ################################
##############################################################################
"""
     CLassical Computation of Shor's Algorithm
"""    

"""
    Function takes a compostite number N and returns its non-trivial factor
"""
def find_factor_of_prime_power(n: int) -> Optional[int]:

    for k in range(2, math.floor(math.log2(n)) + 1):
        c = math.pow(n, 1 / k)
        c1 = math.floor(c)
        if c1 ** k == n:
            return c1
        c2 = math.ceil(c)
        if c2 ** k == n:
            return c2
    return None

"""
    Function checks for prime factors
"""
def find_factor(
    n: int, order_finder: Callable[[int, int], Optional[int]], max_attempts: int = 50
) -> Optional[int]:

    if sympy.isprime(n):
        return None
    if n % 2 == 0:
        return 2
    c = find_factor_of_prime_power(n)
    if c is not None:
        return c
    for _ in range(max_attempts):
        x = random.randint(2, n - 1)
        c = math.gcd(x, n)
        if 1 < c < n:
            return c  # coverage: ignore
        r = order_finder(x, n)
        if r is None:
            continue  # coverage: ignore
        if r % 2 != 0:
            continue  # coverage: ignore
        y = x ** (r // 2) % n
        assert 1 < y < n
        c = math.gcd(y - 1, n)
        if 1 < c < n:
            return c
    return None  # coverage: ignore


##############################################################################
#################################  SECTION 3  ################################
##############################################################################
"""
    Main Function Compiling all parts and implementing Shor's Algorithm
"""
#Algorithm can evaluate all the prime factors of the given number
def main(
    n: int,
    order_finder: Callable[[int, int], Optional[int]] = find_order,
):
    if n < 2:
        raise ValueError('Expected positive integer N must be greater than one.')

    d = find_factor(n, order_finder)

    if d is None:
 #       print(f'No non-trivial factor of {n} found. It is probably a prime.')
      print(f'{n} is Prime')
    else:
        e=n//d
        f=[e]
        g=0
        while(sympy.isprime(e)==False and g!=None):
            g = find_factor(e, order_finder)
            f.append(g)    
            e=e//g

        
                
        if d!=e and e!=f[0]:
            print(f'Prime Factors of {n}: {d}, {e}', end=" ")
            for i in range(1,len(f)):
                if(i!=len(f)-1 and f[i]!=f[1]):
                    print(",",f[i], end=" ")
                elif(f[i]!=f[1]):
                    print(f[i])
                else:
                    print("")
                    break
        elif d!=e:
            print(f'Prime Factors of {n}: {d}, {e}')            
        
        else:
            print(f'Prime Factor of {n}: {d}')

        assert 1 < d < n
        assert n % d == 0

if __name__ == '__main__':

    t=[]
    for i in range(2,36):
        st=time.time()
        main(i,find_order)
        end=time.time()
        t.append(end-st)
        

#Plotting the Runtime Vs Integer Plot 
plt.scatter(list(range(2,36)), t)
plt.title(" Runtime for Prime Factorization Vs Integer ")
plt.xlabel("Value of Integer")
plt.ylabel("Runtime")
plt.show()

2 is Prime
3 is Prime
Prime Factor of 4: 2
5 is Prime
Prime Factors of 6: 2, 3
7 is Prime
Prime Factor of 8: 2
Prime Factor of 9: 3
Prime Factors of 10: 2, 5
11 is Prime
Prime Factors of 12: 2, 3 
13 is Prime
Prime Factors of 14: 2, 7
Prime Factors of 15: 3, 5
Prime Factor of 16: 2
17 is Prime
Prime Factors of 18: 2, 3 
19 is Prime
Prime Factors of 20: 2, 5 
